# A

In [1]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("MyHeroes")
sc = SparkContext(conf = conf)

def parseLine(line):
    fields = line.split(' "')
    heroID = int(fields[0])
    heroName = str(fields[1])[:len(fields[1]) -1]
    return (heroID, heroName)

lines = sc.textFile("hero-name.txt")
rdd = lines.map(parseLine)
rdd.collect()

[(1, '24-HOUR MAN/EMMANUEL'),
 (2, '3-D MAN/CHARLES CHAN'),
 (3, '4-D MAN/MERCURIO'),
 (4, '8-BALL/'),
 (5, 'A'),
 (6, "A'YIN"),
 (7, 'ABBOTT, JACK'),
 (8, 'ABCISSA'),
 (9, 'ABEL'),
 (10, 'ABOMINATION/EMIL BLO'),
 (11, 'ABOMINATION | MUTANT'),
 (12, 'ABOMINATRIX'),
 (13, 'ABRAXAS'),
 (14, 'ADAM 3,031'),
 (15, 'ABSALOM'),
 (16, 'ABSORBING MAN/CARL C'),
 (17, 'ABSORBING MAN | MUTA'),
 (18, 'ACBA'),
 (19, 'ACHEBE, REVEREND DOC'),
 (20, 'ACHILLES'),
 (21, 'ACHILLES II/HELMUT'),
 (22, 'ACROBAT/CARL ZANTE'),
 (23, 'ADAM X'),
 (24, 'ADAMS, CINDY'),
 (25, 'ADAMS, CONGRESSMAN H'),
 (26, 'ADAMS, GEORGE'),
 (27, 'ADAMS, MARTHA'),
 (28, 'ADAMS, NICOLE NIKKI'),
 (29, 'ADAMSON, JASON'),
 (30, 'ADAMSON, REBECCA'),
 (31, 'ADMIRAL PROTOCOL/'),
 (32, 'ADORA'),
 (33, 'ADORA CLONE'),
 (34, 'ADRIA'),
 (35, 'ADVA'),
 (36, 'ADVENT/KYLE GROBE'),
 (37, 'ADVERSARY'),
 (38, 'AEGIS/TREY ROLLINS'),
 (39, 'AENTAROS'),
 (40, 'AFTERLIFE'),
 (41, 'AGAMEMNON'),
 (42, 'AGAMEMNON II/ANDRE'),
 (43, 'AGAMEMNON III/'),
 (44

In [2]:
def parseLine2(line):
    fields = line.split(' ')
    heroID = int(fields[0])
    herolist = []
    for e in fields[1:len(fields)-1]:
        herolist.append(e)
    return (heroID, len(herolist))

lines = sc.textFile("hero-network.txt")
rdd2 = lines.map(parseLine2).reduceByKey(lambda x, y:x+y).map(lambda x: (x[1], x[0]))
rdd2.max()

(1933, 859)

In [3]:
mostPopularName = str(rdd.lookup(859))[2:len(str(rdd.lookup(859))) - 2]
mostPopular = rdd2.max()
print(mostPopularName + " is the most popular superhero, with " + \
      str(mostPopular[0]) + " co-appearances.")

CAPTAIN AMERICA is the most popular superhero, with 1933 co-appearances.


# B

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as func
import collections
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "file:///C:/temp").appName("Customers").getOrCreate()

In [5]:
def mapper(line):
    fields = line.split(',')
    return Row(CustomerID = int(fields[0]), TotalSpent = float(fields[2]))

lines = spark.sparkContext.textFile("customer-orders.csv")
customers = lines.map(mapper)
schemaCustomers = spark.createDataFrame(customers).cache()
schemaCustomers.createOrReplaceTempView("customers")
schemaCustomers.show()

+----------+----------+
|CustomerID|TotalSpent|
+----------+----------+
|        44|     37.19|
|        35|     65.89|
|         2|     40.64|
|        47|     14.98|
|        29|     13.08|
|        91|     24.59|
|        70|     68.68|
|        85|     28.53|
|        53|     83.55|
|        14|      4.32|
|        51|      19.8|
|        42|     57.77|
|         2|     55.77|
|        79|     33.17|
|        50|     23.57|
|        20|      6.49|
|        15|     65.53|
|        44|     99.19|
|         5|     64.18|
|        48|     32.42|
+----------+----------+
only showing top 20 rows



In [8]:
df = schemaCustomers.groupBy("CustomerID").sum('TotalSpent')
df.withColumn('TotalSpent', func.round(df["sum(TotalSpent)"], 2)).drop('sum(TotalSpent)').show()


+----------+----------+
|CustomerID|TotalSpent|
+----------+----------+
|        29|   5032.53|
|        26|    5250.4|
|        65|   5140.35|
|        54|   6065.39|
|        19|   5059.43|
|         0|   5524.95|
|        22|   5019.45|
|         7|   4755.07|
|        77|   4327.73|
|        34|    5330.8|
|        50|   4517.27|
|        94|   4475.57|
|        57|    4628.4|
|        43|   5368.83|
|        32|   5496.05|
|        84|   4652.94|
|        31|   4765.05|
|        98|   4297.26|
|        39|   6193.11|
|        25|   5057.61|
+----------+----------+
only showing top 20 rows

